<h1>Heartbeat Sound Classification with Visual Domain Deep Neural Networks - Dataset B of Pascal heart sound classification challange</h1>

Setting up libraries

In [ ]:
!pip install pydub
!pip install --upgrade scikit-image
!pip install librosa --user
!pip install opencv-python
! apt-get install libsndfile1-dev -y


Import neccesory libraries

In [ ]:
import pandas as pd
import datetime
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import wave
import IPython.display as ipd
import numpy as np
from scipy.signal import butter,filtfilt
import os,shutil
import seaborn as sns
from pydub import AudioSegment
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
curr_dt_time = datetime.datetime.now()


In [ ]:
base_location="Data"

Creating neccesory folders, Add the file inside Data folder

In [ ]:
!mkdir -p "Data/set_b/spectograms/train_data"
!mkdir -p "Data/set_b/spectograms/test_data"

Visualizing the dataset

In [ ]:
setB=pd.read_csv("{0}/set_b.csv".format(base_location))
labelled_setB=setB[setB["label"].notnull()]
## removing the sublabel as thats not required for this study
labelled_setB=labelled_setB[["dataset","fname","label"]]
sns.set(rc={'figure.figsize':(14,5)})
ax = sns.countplot(x ='label', data = labelled_setB)
ax.set(xlabel='Label')

Implementation of Functions - Lowpass filter, includes denoising

In [ ]:
def butter_lowpass_filter(audio_location, cutoff, order):
    wav=get_wav(audio_location)
    x,sr = librosa.load(audio_location,sr=wav.getframerate(),duration=(wav.getnframes()/wav.getframerate()))
    nyq = 0.5 * sr
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, x)
    return y

Implementation of Functions - get the audio duration

In [ ]:
def get_wav(audio_location):
    wav = wave.open(audio_location)
    return wav

def get_audio_duration(audio_location):
    wav = get_wav(audio_location)
    return (wav.getnframes()/wav.getframerate())
     

Slice the audio file

In [ ]:
def get_audio_slice(start_time,end_time,audio_location,part):
    start_ms=start_time*1000
    end_ms=end_time*1000
    newAudio = AudioSegment.from_wav(audio_location)
    newAudio = newAudio[start_ms:end_ms]
    current_file_loc=os.path.split(audio_location)
    exported_file_loc="{0}/split/part_{1}_{2}".format(current_file_loc[0],part,current_file_loc[1])
    newAudio.export(exported_file_loc, format="wav")
    print ("Successfully splitted the {0} as part {1} from {2} to {3} seconds, split file available in {4}".format(current_file_loc[1], part, start_time,end_time, exported_file_loc))
    return exported_file_loc


Split audio files with a length of 3 seconds

In [ ]:
def split_files(audio_location):
    part_factor=3
    tot_duration=get_audio_duration(audio_location) 
    parts=int(tot_duration/part_factor)
    part_file_list=[]
    for part in range(parts):
        start_time=part*part_factor
        end_time=start_time+part_factor
        #print ("Start time of part {0} is {1} and end time is {2} ".format(part,start_time,end_time))  
        file_loc=get_audio_slice(start_time,end_time,audio_location,part)
        part_file_list.append(file_loc)

    return part_file_list

Cleansing the filenames and loading into dataframe

In [ ]:
labelled_setB=setB[setB["label"].notnull()]
labelled_setB=labelled_setB[["dataset","fname","label"]]
## Step 1 : remove "Btraining_" from the file name string for dataset B

labelled_setB['fname'] = labelled_setB['fname'].map(lambda x : x.replace("Btraining_",""))

## As the Audio files has double underscore in the file name for 1st occearnce, upding the index accordingly. Then _noisy file names also getting handled
labelled_setB['fname'] = labelled_setB['fname'].map(lambda x : "{0}/{1}".format(x.split("/")[0],x.split("/")[1].replace("_","__",1)).replace("__noisy","_noisy"))

# updating the duration

labelled_setB['duration'] = labelled_setB['fname'].map(lambda x: get_audio_duration("{0}/{1}".format(base_location,x) ))

Splitting all files of dataset B

In [ ]:
labelled_setB['split_files']=labelled_setB['fname'].map(lambda x: split_files("{0}/{1}".format(base_location,x) ))

Flattening the base dataframe

In [ ]:
pd.set_option('display.max_colwidth', -1)
flatdata = pd.DataFrame([( index, value) for ( index, values)
                         in labelled_setB[ 'split_files' ].iteritems() for value in values],
                             columns = [ 'index', 'split_files']).set_index( 'index' )
  
flattened_labelled_setB = labelled_setB.drop( 'split_files', axis = 1 ).join( flatdata )
#display(flattened_labelled_setB)

Visualizing the flattened dataframe - This will show the data distribution of split files

In [ ]:
## Removing files less than 3 seconds from the index
flattened_labelled_setB=flattened_labelled_setB[flattened_labelled_setB['split_files'].notnull()]
sns.set(rc={'figure.figsize':(14,5)})
ax = sns.countplot(x ='label', data = flattened_labelled_setB, palette=["#ab594f","#d19421","green"])

count = flattened_labelled_setB.groupby(['label'])['fname'].count().values

pos = range(len(count))


for tick in pos:
  ax.text(pos[tick],count[tick]+20, count[tick], horizontalalignment='center', size='small', color='black', weight='regular')
ax.set(xlabel='Cardiovascular sound category')
ax.set(ylabel='Count')
ax.set(title="Dataset B")

Setting the denoising cutoff and denoising the audio data

In [ ]:
cutoff=192 #Hz
order=1
#butter_lowpass_filter(str(x),cutoff,order)
flattened_labelled_setB["denoised_signal"]=flattened_labelled_setB['split_files'].map(lambda x: butter_lowpass_filter(str(x),cutoff,order) )

In [ ]:
flattened_labelled_setB.to_csv("Data/set_b/denoise_split_master.csv")

Randomly shuffling the denoised audio signals

In [ ]:
flattened_labelled_setB=shuffle(flattened_labelled_setB)

Splitting into train and test data

In [ ]:
train, test = train_test_split(flattened_labelled_setB, test_size=0.3)
print(len(train))
print(len(test))

## Removing files less than 3 seconds from the index
fig, ax = plt.subplots(1,2, sharey=True,figsize=(15, 5))
#sns.set(rc={'figure.figsize':(14,5)})
ax_1 = sns.countplot(x ='label', data = train,ax=ax[0])
ax_1.set(xlabel='TRAIN')
ax_2 = sns.countplot(x ='label', data = test,ax=ax[1])
ax_2.set(xlabel='TEST')

Implementation of specAugment - reference https://github.com/KimJeongSun/SpecAugment_numpy_scipy

In [ ]:
import time
import random
import argparse
import sys

import numpy as np
import numpy.linalg as nl
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy import interpolate
from scipy import signal
from scipy.io import wavfile
from scipy.fftpack import dct,idct
from scipy.spatial.distance import pdist, cdist, squareform
import skimage.io


def plot_spec(spec,out):
    librosa.display.specshow(spec, fmax=8000)
    plt.savefig(out)
    plt.cla()


def makeT(cp):
    # cp: [K x 2] control points
    # T: [(K+3) x (K+3)]
    K = cp.shape[0]
    T = np.zeros((K+3, K+3))
    T[:K, 0] = 1
    T[:K, 1:3] = cp
    T[K, 3:] = 1
    T[K+1:, 3:] = cp.T
    R = squareform(pdist(cp, metric='euclidean'))
    R = R * R
    R[R == 0] = 1 # a trick to make R ln(R) 0
    R = R * np.log(R)
    np.fill_diagonal(R, 0)
    T[:K, 3:] = R
    return T

def liftPts(p, cp):
    # p: [N x 2], input points
    # cp: [K x 2], control points
    # pLift: [N x (3+K)], lifted input points
    N, K = p.shape[0], cp.shape[0]
    pLift = np.zeros((N, K+3))
    pLift[:,0] = 1
    pLift[:,1:3] = p
    R = cdist(p, cp, 'euclidean')
    R = R * R
    R[R == 0] = 1
    R = R * np.log(R)
    pLift[:,3:] = R
    return pLift
    
def specAug(audio,sampling_rate,num,out):
    time_sum = 0

    #audio, sampling_rate = librosa.load(args.inpu,sr=t)
    spec = librosa.feature.melspectrogram(y=audio,sr=sampling_rate,n_fft=200, hop_length=4)
    spec = librosa.power_to_db(spec,ref=np.max)
    
    print("start to SpecAugment %d times" % num)
    for n in range(num): 
        start = time.time()
        W=40
        T=30
        F=13
        mt=2
        mf=2

        # Nframe : number of spectrum frame
        Nframe = spec.shape[1]
        # Nbin : number of spectrum freq bin
        Nbin = spec.shape[0]
        # check input length
        if Nframe < W*2+1:
            W = int(Nframe/4)
        if Nframe < T*2+1:
            T = int(Nframe/mt)
        if Nbin < F*2+1:
            F = int(Nbin/mf)

        # warping parameter initialize
        w = random.randint(-W,W)
        center = random.randint(W,Nframe-W)

        src = np.asarray([[ float(center),  1], [ float(center),  0], [ float(center),  2], [0, 0], [0, 1], [0, 2], [Nframe-1, 0], [Nframe-1, 1], [Nframe-1, 2]])
        dst = np.asarray([[ float(center+w),  1], [ float(center+w),  0], [ float(center+w),  2], [0, 0], [0, 1], [0, 2], [Nframe-1, 0], [Nframe-1, 1], [Nframe-1, 2]])
        #print(src,dst)

        # source control points
        xs, ys = src[:,0],src[:,1]
        cps = np.vstack([xs, ys]).T
        # target control points
        xt, yt = dst[:,0],dst[:,1]
        # construct TT
        TT = makeT(cps)

        # solve cx, cy (coefficients for x and y)
        xtAug = np.concatenate([xt, np.zeros(3)])
        ytAug = np.concatenate([yt, np.zeros(3)])
        cx = nl.solve(TT, xtAug) # [K+3]
        cy = nl.solve(TT, ytAug)

        # dense grid
        x = np.linspace(0, Nframe-1,Nframe)
        y = np.linspace(1,1,1)
        x, y = np.meshgrid(x, y)

        xgs, ygs = x.flatten(), y.flatten()

        gps = np.vstack([xgs, ygs]).T

        # transform
        pgLift = liftPts(gps, cps) # [N x (K+3)]
        xgt = np.dot(pgLift, cx.T)     
        spec_warped = np.zeros_like(spec)
        for f_ind in range(Nbin):
            spec_tmp = spec[f_ind,:]
            func = interpolate.interp1d(xgt, spec_tmp,fill_value="extrapolate")
            xnew = np.linspace(0, Nframe-1,Nframe)
            spec_warped[f_ind,:] = func(xnew)

        # sample mt of time mask ranges
        t = np.random.randint(T-1, size=mt)+1
        # sample mf of freq mask ranges
        f = np.random.randint(F-1, size=mf)+1
        # mask_t : time mask vector
        mask_t = np.ones((Nframe,1))
        ind = 0
        t_tmp = t.sum() + mt
        for _t in t:
            k = random.randint(ind,Nframe-t_tmp)
            mask_t[k:k+_t] = 0
            ind = k+_t+1
            t_tmp = t_tmp - (_t+1)
        mask_t[ind:] = 1

        # mask_f : freq mask vector
        mask_f = np.ones((Nbin,1))
        ind = 0
        f_tmp = f.sum() + mf
        for _f in f:
            k = random.randint(ind,Nbin-f_tmp)
            mask_f[k:k+_f] = 0
            ind = k+_f+1
            f_tmp = f_tmp - (_f+1)
        mask_f[ind:] = 1

        # calculate mean
        mean = np.mean(spec_warped)

        # make spectrum to zero mean
        spec_zero = spec_warped-mean

        spec_masked = ((spec_zero * mask_t.T) * mask_f) + mean
    #     spec_masked = ((spec_zero * mask_t).T * mask_f).T

        end = time.time()
        time_sum += (end - start)  
        if n == 0:
          plot_spec(spec,"{0}_orginal.png".format(out))
          plot_spec(spec_warped,"{0}_wrapped.png".format(out))
        plot_spec(spec_masked,"{0}_masked_{1}.png".format(out,n))
    print("whole processing time : %.4f second" % (time_sum))   
    print("average processing time : %.2f ms" % (time_sum*1000/num))


Generate Spectrogram and Augment with timewrapped and frequency masked Spectrogram

In [ ]:
def augment_spectrograms(audio,sr,filename,aug_num,step):
  file_list=[]
  filename=filename.replace("split","spectograms/{0}_data".format(step))
  filename=filename.replace(".wav","")
  specAug(audio,sr,aug_num,filename)
  file_list.append("{0}_orginal.png".format(filename))
  if step =='train':
    file_list.append("{0}_wrapped.png".format(filename))
    for n in range (aug_num):
        file_list.append("{0}_masked_{1}.png".format(filename,n))   
  return file_list

Generating training spectogram images

In [ ]:
sr=4000
data=list((map(lambda x,y: augment_spectrograms(x,sr,y,1,'train'),train['denoised_signal'],train['split_files'])))


Generating only the test spectograms (without augmentation)

In [ ]:
test_data=list((map(lambda x,y: augment_spectrograms(x,sr,y,1,'test'),test['denoised_signal'],test['split_files'])))

Adding file path information into existing pandas dataframe

In [ ]:
train=train.assign(augmented_files = data)
test=test.assign(augmented_files = test_data)

In [ ]:
test

Flattening both train and test datasets

In [ ]:
flat_aug_data = pd.DataFrame([( index, value) for ( index, values)
                         in train[ 'augmented_files' ].iteritems() for value in values],
                             columns = [ 'index', 'augmented_files']).set_index( 'index' )
  
train_aug = train.drop( 'augmented_files', axis = 1 ).join( flat_aug_data )


In [ ]:
flat_aug_data = pd.DataFrame([( index, value) for ( index, values)
                         in test[ 'augmented_files' ].iteritems() for value in values],
                             columns = [ 'index', 'augmented_files']).set_index( 'index' )
  
test_aug = test.drop( 'augmented_files', axis = 1 ).join( flat_aug_data )


Saving the train and test file location and label information. Also visualzing the data distribution

In [ ]:
train_aug_file_label=train_aug[["augmented_files","label"]]
train_aug_file_label.to_csv("set_b_train_file_to_label.csv")
ax_1 = sns.countplot(x ='label', data = train_aug_file_label)
ax_1.set(xlabel='TRAIN')

In [ ]:
test_aug_file_label=test_aug[["augmented_files","label"]]
test_aug_file_label.to_csv("set_b_test_file_to_label_v1.csv")
ax_1 = sns.countplot(x ='label', data = test_aug_file_label)
ax_1.set(xlabel='TEST')

Plot images function

In [ ]:
# function to plot n images using subplots
import cv2 as cv
from skimage.transform import rescale, resize
from matplotlib import pyplot as plt

        
def plot_image(images, captions=None, cmap=None ):
    f, axes = plt.subplots(1, len(images), sharey=True)
    f.set_figwidth(15)
    for ax,image in zip(axes,images):
        ax.imshow(image, cmap)

Scaling Images

In [ ]:
def image_scale(imagePath, image_height, image_width):
    image = cv.imread(imagePath)
    image = cv.resize(image, (image_height, image_width))                    
    return image


Resizing spectrograms into 128*128 image

In [ ]:
import pandas as pd
data_mapping=pd.read_csv("set_b_train_file_to_label.csv")
data_mapping["image"]=data_mapping['augmented_files'].map(lambda x : image_scale(x,128,128))

Shuffling dataset and Normalzing pixels

In [ ]:
from sklearn.utils import shuffle
import numpy as np

#=data_mapping['image'].map(lambda x : x/255.0)
data_mapping=shuffle(data_mapping)
data=np.array(list(data_mapping["image"]), dtype="float") / 255.0

Encoding labales as required

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
le = LabelEncoder()
labels=list(data_mapping["label"])

train_label = le.fit_transform(labels)
labels = to_categorical(train_label,3)

Counting each category for weight calculation

In [ ]:
murmur= len(list(filter(lambda x : x==1 , train_label)))
normal= len(list(filter(lambda x : x==2 , train_label)))
extrasystole=len(list(filter(lambda x : x==0 , train_label)))
extrasystole+normal+murmur

In [ ]:
train_label

Training vs Validation split

In [ ]:
from sklearn.model_selection import train_test_split

(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)

Base Image generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator()
val_aug = ImageDataGenerator()

Loading test data for evaluation and pre-prprocess

In [ ]:
import pandas as pd
#import librosa
#import librosa.display
import matplotlib.pyplot as plt
import wave
import IPython.display as ipd
import numpy as np
from scipy.signal import butter,filtfilt
import os,shutil
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

cutoff=192 #Hz
order=1
#butter_lowpass_filter(str(x),cutoff,order)

import pandas as pd
test_data_mapping=pd.read_csv("""set_b_test_file_to_label_v1.csv""")
test_data_mapping["image"]=test_data_mapping['augmented_files'].map(lambda x : image_scale(x,128,128))
from sklearn.utils import shuffle
import numpy as np

#=data_mapping['image'].map(lambda x : x/255.0)
test_data_mapping=shuffle(test_data_mapping)
test_data=np.array(list(test_data_mapping["image"]), dtype="float") / 255.0

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
#le = LabelEncoder()
test_labels=list(test_data_mapping["label"])

test_labels = le.transform(test_labels)
test_labels = to_categorical(test_labels,3)

Create required callbacks

In [ ]:
def create_callback(model_type, reduce_plateau_factor, patience_val,save_best_only):
    from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
    model_name = 'models/'
    
    if not os.path.exists(model_name):
         os.mkdir(model_name)
        
    filepath = model_name + model_type+ '-model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=save_best_only, save_weights_only=False, mode='auto', period=1)


    LR = ReduceLROnPlateau(monitor='val_loss', factor=reduce_plateau_factor, patience=patience_val, verbose = 1)
    callbacks_list = [checkpoint, LR]
    return callbacks_list

Training with MobileNet

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, MobileNet,ResNet50
from tensorflow.keras.applications import ResNet50
import tensorflow.keras
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM, Reshape, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

#from keras.metrics import TruePositives
#from tensorflow.keras.layers.convolutional import Conv2D, MaxPooling2D 

lr=0.01; momentum_val=0.9;
EPOCHS=20
BS=8

# initialize the optimizer and model
print("[INFO] compiling model...")
#opt = SGD(lr=lr, momentum=0.9, decay=lr / EPOCHS)
opt = Adam(lr=lr, decay=lr / EPOCHS)
pretrained_model = MobileNet (
        include_top=False,
        input_shape=(128,128,3),
        weights='imagenet'
     )
#pretrained_model.trainable = True


train_generator = aug.flow(trainX, trainY, batch_size=BS)

validation_generator = val_aug.flow(testX, testY,batch_size=BS)




model = Sequential()
model.add(pretrained_model)
#model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64, activation = "relu")) # fully connected
model.add(Dense(3, activation='softmax'))

weight_for_extrasystole = (extrasystole)/(train_label.size) 
weight_for_murmur = (murmur)/(train_label.size) 
weight_for_normal = (normal)/(train_label.size) 

class_weight = {0: weight_for_extrasystole, 1: weight_for_murmur, 2: weight_for_normal}


model_type="MobileNet" ; reduce_plateau_factor=0.2; patience_val=5
callbacks_list = create_callback(model_type, reduce_plateau_factor, patience_val, save_best_only = True)


model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["categorical_accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC()])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS,epochs=EPOCHS,
                        validation_data=(testX, testY),class_weight=class_weight,callbacks=callbacks_list,shuffle = True)

Evaluation with MobileNet

In [ ]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot

from tensorflow.keras.models import load_model

model = load_model("models/MobileNet-model-00015-0.02524-0.92047-0.22122-0.93095.h5")


score, acc,precision,recall,auc = model.evaluate(test_data, test_labels, batch_size=8)



Training with InceptionResNetV2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, MobileNet,ResNet50,InceptionResNetV2
from tensorflow.keras.applications import ResNet50
import tensorflow.keras
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM, Reshape, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
 

lr=0.001; momentum_val=0.9;
EPOCHS=10
BS=8

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=lr, momentum=0.9, decay=lr / EPOCHS)
#opt = Adam(lr=lr, decay=lr / EPOCHS)
pretrained_model = InceptionResNetV2 (
        include_top=False,
        input_shape=(128,128,3),
        weights='imagenet'
     )
#pretrained_model.trainable = True


train_generator = aug.flow(trainX, trainY, batch_size=BS)

validation_generator = val_aug.flow(testX, testY,batch_size=BS)




model = Sequential()
model.add(pretrained_model)
#model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64, activation = "relu")) # fully connected
model.add(Dense(3, activation='softmax'))

weight_for_extrasystole = (extrasystole)/(train_label.size) 
weight_for_murmur = (murmur)/(train_label.size) 
weight_for_normal = (normal)/(train_label.size) 

class_weight = {0: weight_for_extrasystole, 1: weight_for_murmur, 2: weight_for_normal}


model_type="InceptionResNetV2" ; reduce_plateau_factor=0.2; patience_val=5
callbacks_list = create_callback(model_type, reduce_plateau_factor, patience_val, save_best_only = True)


model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["categorical_accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC()])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS,epochs=EPOCHS,
                        validation_data=(testX, testY),class_weight=class_weight,callbacks=callbacks_list,shuffle = True)

Eveluation with InceptionResNetV2

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("models/InceptionResNetV2-model-00005-0.01576-0.95970-0.14147-0.96905.h5")

test_aug = ImageDataGenerator()


score, acc,precision,recall,auc = model.evaluate(test_data, test_labels, batch_size=8)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, MobileNet,ResNet50,InceptionResNetV2,MobileNetV2
from tensorflow.keras.applications import ResNet50
import tensorflow.keras
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM, Reshape, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

#from keras.metrics import TruePositives
#from tensorflow.keras.layers.convolutional import Conv2D, MaxPooling2D 

lr=0.0001; momentum_val=0.9;
EPOCHS=10
BS=8

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=lr, momentum=0.9, decay=lr / EPOCHS)
#opt = Adam(lr=lr, decay=lr / EPOCHS)
pretrained_model = MobileNetV2 (
        include_top=False,
        input_shape=(128,128,3),
        weights='imagenet'
     )
#pretrained_model.trainable = True


train_generator = aug.flow(trainX, trainY, batch_size=BS)

validation_generator = val_aug.flow(testX, testY,batch_size=BS)




model = Sequential()
model.add(pretrained_model)
#model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64, activation = "relu")) # fully connected
model.add(Dense(3, activation='softmax'))

weight_for_extrasystole = (extrasystole)/(train_label.size) 
weight_for_murmur = (murmur)/(train_label.size) 
weight_for_normal = (normal)/(train_label.size) 

class_weight = {0: weight_for_extrasystole, 1: weight_for_murmur, 2: weight_for_normal}


model_type="MobileNetV2" ; reduce_plateau_factor=0.2; patience_val=5
callbacks_list = create_callback(model_type, reduce_plateau_factor, patience_val, save_best_only = True)


model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["categorical_accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC()])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS,epochs=EPOCHS,
                        validation_data=(testX, testY),class_weight=class_weight,callbacks=callbacks_list,shuffle = True)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("models/MobileNetV2-model-00007-0.02528-0.93401-0.29812-0.90041.h5")

test_aug = ImageDataGenerator()


score, acc,precision,recall,auc = model.evaluate(test_data, test_labels, batch_size=8)



In [ ]:
fig, ax = plt.subplots(1,1, sharey=True,figsize=(15, 5))

plt.plot(H.history['auc_3'])
plt.plot(H.history['val_auc_3'])
plt.title('Dataset B - Training History - MobileNetV2')
plt.ylabel('ROC-AUC')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Training with ResNet152V2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, MobileNet,ResNet50,InceptionResNetV2,MobileNetV2,EfficientNetB5,ResNet152V2
from tensorflow.keras.applications import ResNet50
import tensorflow.keras
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM, Reshape, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping


lr=0.001; momentum_val=0.9;
EPOCHS=20
BS=8

# initialize the optimizer and model
print("[INFO] compiling model...")
#opt = SGD(lr=lr, momentum=0.9, decay=lr / EPOCHS)
opt = Adam(lr=lr, decay=lr / EPOCHS)
pretrained_model = ResNet152V2 (
        include_top=False,
        input_shape=(128,128,3),
        weights='imagenet'
     )
#pretrained_model.trainable = True


train_generator = aug.flow(trainX, trainY, batch_size=BS)

validation_generator = val_aug.flow(testX, testY,batch_size=BS)




model = Sequential()
model.add(pretrained_model)
#model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64, activation = "relu")) # fully connected
model.add(Dense(3, activation='softmax'))

weight_for_extrasystole = (extrasystole)/(train_label.size) 
weight_for_murmur = (murmur)/(train_label.size) 
weight_for_normal = (normal)/(train_label.size) 

class_weight = {0: weight_for_extrasystole, 1: weight_for_murmur, 2: weight_for_normal}


model_type="ResNet152V2" ; reduce_plateau_factor=0.2; patience_val=5
callbacks_list = create_callback(model_type, reduce_plateau_factor, patience_val, save_best_only = True)


model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["categorical_accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC()])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS,epochs=EPOCHS,
                        validation_data=(testX, testY),class_weight=class_weight,callbacks=callbacks_list,shuffle = True)

In [ ]:
fig, ax = plt.subplots(1,1, sharey=True,figsize=(15, 5))

plt.plot(H.history['auc_5'])
plt.plot(H.history['val_auc_5'])
plt.title('Dataset B - Training History - ResNet152V2')
plt.ylabel('ROC-AUC')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

ResNet152V2 evaluation

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("models/ResNet152V2-model-00017-0.01505-0.94254-0.21193-0.94074.h5")

score, acc,precision,recall,auc = model.evaluate(test_data, test_labels, batch_size=8)



Training with Xception

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, MobileNet,ResNet50,InceptionResNetV2,MobileNetV2,EfficientNetB5,ResNet152V2,Xception
from tensorflow.keras.applications import ResNet50
import tensorflow.keras
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM, Reshape, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping


lr=0.001; momentum_val=0.9;
EPOCHS=7
BS=8

# initialize the optimizer and model
print("[INFO] compiling model...")
#opt = SGD(lr=lr, momentum=0.9, decay=lr / EPOCHS)
opt = Adam(lr=lr, decay=lr / EPOCHS)
pretrained_model = Xception (
        include_top=False,
        input_shape=(128,128,3),
        weights='imagenet'
     )


train_generator = aug.flow(trainX, trainY, batch_size=BS)

validation_generator = val_aug.flow(testX, testY,batch_size=BS)




model = Sequential()
model.add(pretrained_model)
#model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64, activation = "relu")) # fully connected
model.add(Dense(3, activation='softmax'))

weight_for_extrasystole = (extrasystole)/(train_label.size) 
weight_for_murmur = (murmur)/(train_label.size) 
weight_for_normal = (normal)/(train_label.size) 

class_weight = {0: weight_for_extrasystole, 1: weight_for_murmur, 2: weight_for_normal}


model_type="Xception" ; reduce_plateau_factor=0.2; patience_val=5
callbacks_list = create_callback(model_type, reduce_plateau_factor, patience_val, save_best_only = True)


model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["categorical_accuracy",tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC()])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS,epochs=EPOCHS,
                        validation_data=(testX, testY),class_weight=class_weight,callbacks=callbacks_list,shuffle = True)

Evaluation with Xception Network

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("models/Xception-model-00005-0.02759-0.92100-0.18878-0.92222.h5")


score, acc,precision,recall,auc = model.evaluate(test_data, test_labels, batch_size=8)

